# Lyric Sandbox verification

This notebook will be showing the capabilities of the Lyric Sandbox environment. 
It will run Python code and JavaScript code in the sandbox environment which is isolated from the host environment.

If you not familiar with the lyric, please read this notebook first: [Quick Start](lyric_quick_start.ipynb)

## 1. File System Isolation

The sandbox environment runs in webassembly environment, we can control the file system access of the sandbox environment.

It can't access any file in the host environment by default.

Let's try to access a file in the host environment.

In [1]:
# First start lyric driver

from lyric import DefaultLyricDriver

lcd = DefaultLyricDriver(host="127.0.0.1", log_level="ERROR")
lcd.start()

# Load workers(default: Python, JavaScript)
await lcd.lyric.load_default_workers()

In [2]:
# List the files in the root directory

python_code = """
import os

# List the files in the root directory
root = os.listdir('/')
print("Files in the root directory:", root)
"""

py_res = await lcd.exec(python_code, "python")
assert py_res.exit_code !=0, "Python code should fail"

print(py_res)
print(f"logs:\n{py_res.logs}")

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code='\\nimport os\\n\\n# List the files in the root directory\\nroot = os.listdir(\\'/\\')\\nprint(\"Files in the root directory:\", root)\\n')"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/'"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] test_result: b'{\"content\":\"Execute script successfully\",\"exit_code\":1,\"lang\":\"Python\",\"protocol\":1,\"stderr\":\"\",\"stdout\":\"[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: \\'/\\'\\\\n\",\"success\":false}'"
CodeResult(exit_code=1, stdout="[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/'\n", stderr='', output=None)
logs:
[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/'



As you can see, the exit code is 1, and the output is 
```
[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/'
```

In [3]:
# Create directory

python_code = """
import os

# Create a directory
os.mkdir('/test')
"""

py_res = await lcd.exec(python_code, "python")
assert py_res.exit_code !=0, "Python code should fail"
print(py_res)
print(f"logs:\n{py_res.logs}")

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code=\"\\nimport os\\n\\n# Create a directory\\nos.mkdir('/test')\\n\")"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/test'"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] test_result: b'{\"content\":\"Execute script successfully\",\"exit_code\":1,\"lang\":\"Python\",\"protocol\":1,\"stderr\":\"\",\"stdout\":\"[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: \\'/test\\'\\\\n\",\"success\":false}'"
CodeResult(exit_code=1, stdout="[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/test'\n", stderr='', output=None)
logs:
[Python-InterpreterTask] Exception: [Errno 44] No such file or directory: '/test'



## 2. Network Isolation

The sandbox environment can't access the network by default because the network is disabled in the sandbox environment.

Let's try to create a socket to access the network.

In [4]:
# Create a socket to connect to a server

py_client_code = """
import socket

def start_client():
    # Create TCP/IP socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Connect the socket to the server
    server_address = ('127.0.0.1', 12345)
    print(f"Connecting to {server_address}")
    client_socket.connect(server_address)
    
    try:
        # Send data
        message = "Hello, Server!"
        print(f"Sending: {message}")
        client_socket.sendall(message.encode())
        
        # Receive data
        data = client_socket.recv(1024).decode()
        print(f"Received: {data}")
        
    finally:
        # Close the connection
        print("Closing connection")
        client_socket.close()
start_client()
"""

py_res = await lcd.exec(py_client_code, "python")
assert py_res.exit_code !=0, "Python code should fail"
print(py_res)

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code='\\nimport socket\\n\\ndef start_client():\\n    # Create TCP/IP socket\\n    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\\n    \\n    # Connect the socket to the server\\n    server_address = (\\'127.0.0.1\\', 12345)\\n    print(f\"Connecting to {server_address}\")\\n    client_socket.connect(server_address)\\n    \\n    try:\\n        # Send data\\n        message = \"Hello, Server!\"\\n        print(f\"Sending: {message}\")\\n        client_socket.sendall(message.encode())\\n        \\n        # Receive data\\n        data = client_socket.recv(1024).decode()\\n        print(f\"Received: {data}\")\\n        \\n    finally:\\n        # Close the connection\\n        print(\"Closing connection\")\\n        client_socket.close()\\nstart_client()\\n')"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "C

As you can see, the exit code is 1, and the output is 
```python
CodeResult(exit_code=1, stdout="Connecting to ('127.0.0.1', 12345)\n[Python-InterpreterTask] Exception: [Errno 2] Permission denied\n", stderr='', output=None)
```

In [8]:
# Create a server to accept connections

py_server_code = """
import socket

def start_server():
    # Create TCP/IP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind the address and port
    server_address = ('127.0.0.1', 12345)
    server_socket.bind(server_address)
    
    # Begin listening, maximum queue connection is 1
    server_socket.listen(1)
    print(f"Server listening on {server_address}")
    
    while True:
        # Wait for connection
        print("Waiting for connection...")
        client_socket, client_address = server_socket.accept()
        try:
            print(f"Connected to {client_address}")
            
            # Receive data
            data = client_socket.recv(1024).decode()
            print(f"Received: {data}")
            
            # Send response
            response = f"Server received: {data}"
            client_socket.sendall(response.encode())
            
        finally:
            # Close the connection
            client_socket.close()
start_server()
"""

py_res = await lcd.exec(py_server_code, "python")
assert py_res.exit_code !=0, "Python code should fail"
print(py_res)

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code='\\nimport socket\\n\\ndef start_server():\\n    # Create TCP/IP socket\\n    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\\n    \\n    # Bind the address and port\\n    server_address = (\\'127.0.0.1\\', 12345)\\n    server_socket.bind(server_address)\\n    \\n    # Begin listening, maximum queue connection is 1\\n    server_socket.listen(1)\\n    print(f\"Server listening on {server_address}\")\\n    \\n    while True:\\n        # Wait for connection\\n        print(\"Waiting for connection...\")\\n        client_socket, client_address = server_socket.accept()\\n        try:\\n            print(f\"Connected to {client_address}\")\\n            \\n            # Receive data\\n            data = client_socket.recv(1024).decode()\\n            print(f\"Received: {data}\")\\n            \\n            # Send response\\n

## 3. Process And Memory Isolation

In [5]:
py_code = """
import multiprocessing
import ctypes

# Try to spawn a process
def spawn_process():
    print('Spawned!')
p = multiprocessing.Process(target=spawn_process)
p.start()

# Try to access memory directly
libc = ctypes.CDLL('libc.so.6')
"""

py_res = await lcd.exec(py_code, "python")
assert py_res.exit_code !=0, "Python code should fail"
print(py_res)

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code=\"\\nimport multiprocessing\\nimport ctypes\\n\\n# Try to spawn a process\\ndef spawn_process():\\n    print('Spawned!')\\np = multiprocessing.Process(target=spawn_process)\\np.start()\\n\\n# Try to access memory directly\\nlibc = ctypes.CDLL('libc.so.6')\\n\")"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] Exception: No module named 'ctypes'"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] test_result: b'{\"content\":\"Execute script successfully\",\"exit_code\":1,\"lang\":\"Python\",\"protocol\":1,\"stderr\":\"\",\"stdout\":\"[Python-InterpreterTask] Exception: No module named \\'ctypes\\'\\\\n\",\"success\":false}'"
CodeResult(exit_code=1, stdout="[Python-InterpreterTask] Exception: No module named 'ctypes'\n", stderr='', output=None)


## 4. Time And Scheduler Attack Verification

In [9]:
py_code = """
import threading
import time

def delayed_attack():
    time.sleep(10)  
    __import__('os').system('echo "Delayed attack!"')

thread = threading.Thread(target=delayed_attack)
thread.start()
"""

py_res = await lcd.exec(py_code, "python")
assert py_res.exit_code !=0, "Python code should fail"
print(py_res)

["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] script: InterpreterRequest(protocol=1, lang='PYTHON', code='\\nimport threading\\nimport time\\n\\ndef delayed_attack():\\n    time.sleep(10)  \\n    __import__(\\'os\\').system(\\'echo \"Delayed attack!\"\\')\\n\\nthread = threading.Thread(target=delayed_attack)\\nthread.start()\\n')"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] Exception: can't start new thread"
["local_d112c954598914a1@343f4169-9f13-4856-aee3-0c30f6af6551"]: "[Python-InterpreterTask] test_result: b'{\"content\":\"Execute script successfully\",\"exit_code\":1,\"lang\":\"Python\",\"protocol\":1,\"stderr\":\"\",\"stdout\":\"[Python-InterpreterTask] Exception: can\\'t start new thread\\\\n\",\"success\":false}'"
CodeResult(exit_code=1, stdout="[Python-InterpreterTask] Exception: can't start new thread\n", stderr='', output=None)
